In [1]:
import numpy as np  # Librería para aplicar álgebra lineal
import pandas as pd # Para manejar los datos (datasets)

import matplotlib.pyplot as plt # Para las visualizaciones
import seaborn as sns # Visualizaciones más fancy

from sklearn import preprocessing # Para el procesador de los datos
from sklearn.preprocessing import Imputer # Para adoptar una estrategia para los missing values
from sklearn.preprocessing import LabelEncoder as Codificar # Para codificar variables categóricas
from sklearn.preprocessing import OneHotEncoder # Para pasar el LaberEncoder vector a OneHot matriz
from sklearn.preprocessing import MinMaxScaler # Para realizar el escalado en escala (0-1)
from sklearn.model_selection import train_test_split as Separar # Para dividir en los 2 conjuntos
#from statsmodels.tools.eval_measures import rmse # Para calcular el error
from sklearn.metrics import confusion_matrix as CM # Para construir la matriz de confusión
from matplotlib.colors import ListedColormap as Colors # Para pintar las regiones en Clasificación

In [2]:
#Se especifican los tipo de datos para mejorar la performance
eventos = pd.read_csv('../TP2018/fiuba-trocafone-tp2-final-set/events_up_to_01062018.csv', 
                      dtype={"timestamp": object,#intenté hacerla datetime y no anduvo 
                             "event": 'category',
                             "person":object,
                             "url":object,
                             "sku":object,
                             "model":object,
                             "condition":'category',
                             "storage":object,
                             "color":'category',
                             "skus":object,
                             "search_term":object,
                             "staticpage":object,
                             "campaign_source":object,
                             "search_engine":object,
                             "channel":object,
                             "new_vs_returning":'category',
                             "city":object,
                             "region":object,
                             "country":object,
                             "device_type":object,
                             "screen_resolution":object,
                             "operating_system_version":object,
                             "browser_version":object})
etiquetas = pd.read_csv('../TP2018/fiuba-trocafone-tp2-final-set/labels_training_set.csv',
                        dtype={"person":object, "label":bool})

In [3]:
etiquetas['label'].value_counts()

False    18434
True       980
Name: label, dtype: int64

In [4]:
eventos.head(3)

,timestamp,event,person,url,sku,model,condition,storage,color,skus,...,search_engine,channel,new_vs_returning,city,region,country,device_type,screen_resolution,operating_system_version,browser_version
0,2018-05-18 00:11:59,viewed product,4886f805,NaN,9288.0,Samsung Galaxy J7 Prime,Excelente,32GB,Dourado,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2018-05-18 00:11:27,viewed product,ad93850f,NaN,304.0,iPhone 5s,Muito Bom,32GB,Cinza espacial,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2018-05-18 00:11:16,viewed product,0297fc1e,NaN,6888.0,iPhone 6S,Muito Bom,64GB,Prateado,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
len(eventos['person'].unique()) - len(etiquetas)

19415

In [6]:
print(etiquetas['label'].value_counts())
print(etiquetas['label'].value_counts(normalize=True))

False    18434
True       980
Name: label, dtype: int64
False    0.949521
True     0.050479
Name: label, dtype: float64


In [7]:
eventos['timestamp'] =  pd.to_datetime(eventos['timestamp'])
eventos[['marca','modelo']] = eventos['model'].dropna().str.split(' ',n=1,expand=True)
eventos['weekday']=eventos['timestamp'].dt.day_name()
eventos['hour']=eventos['timestamp'].dt.hour
eventos['month']=eventos['timestamp'].dt.month
eventos['day']=eventos['timestamp'].dt.day

In [8]:
print('Cantidad de meses distintos:',len(eventos['month'].value_counts()))
eventos['month'].value_counts()

Cantidad de meses distintos: 5


5    1713920
4     309849
3     193790
2      73541
1      50581
Name: month, dtype: int64

In [9]:
eventos.groupby('person')['month'].value_counts().unstack().reset_index().fillna(0).head()

month,person,1,2,3,4,5
0,0008ed71,0.0,0.0,0.0,0.0,6.0
1,00091926,0.0,0.0,0.0,0.0,448.0
2,00091a7a,0.0,0.0,10.0,0.0,0.0
3,000ba417,0.0,0.0,0.0,0.0,206.0
4,000c79fe,0.0,0.0,0.0,0.0,17.0


In [10]:
eventosGrp  = eventos.groupby('person')['event'].value_counts().unstack().reset_index().fillna(0)
eventosGrp.head(3)

event,person,ad campaign hit,brand listing,checkout,conversion,generic listing,lead,search engine hit,searched products,staticpage,viewed product,visited site
0,0008ed71,0.0,0.0,3.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,2.0
1,00091926,15.0,25.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,372.0,34.0
2,00091a7a,1.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,1.0


In [11]:
mesesGrp = eventos.groupby('person')['month'].value_counts().unstack().reset_index().fillna(0)
mesesGrp.rename({1:'enero',2:'febreo',3:'marzo',4:'abril',5:'mayo'},inplace=True, axis=1)
mesesGrp.head(3)

month,person,enero,febreo,marzo,abril,mayo
0,0008ed71,0.0,0.0,0.0,0.0,6.0
1,00091926,0.0,0.0,0.0,0.0,448.0
2,00091a7a,0.0,0.0,10.0,0.0,0.0


In [12]:
#Agrupamos los primeros features, de mes y evento
clientesGrp = pd.merge(eventosGrp,mesesGrp,on='person')
clientesGrp.head()

,person,ad campaign hit,brand listing,checkout,conversion,generic listing,lead,search engine hit,searched products,staticpage,viewed product,visited site,enero,febreo,marzo,abril,mayo
0,0008ed71,0.0,0.0,3.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,6.0
1,00091926,15.0,25.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,372.0,34.0,0.0,0.0,0.0,0.0,448.0
2,00091a7a,1.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,1.0,0.0,0.0,10.0,0.0,0.0
3,000ba417,1.0,24.0,6.0,1.0,14.0,0.0,1.0,0.0,0.0,153.0,6.0,0.0,0.0,0.0,0.0,206.0
4,000c79fe,1.0,0.0,1.0,0.0,1.0,0.0,1.0,9.0,0.0,3.0,1.0,0.0,0.0,0.0,0.0,17.0


In [13]:
marcasGrp = eventos.groupby('person')['marca'].value_counts().unstack().fillna(0).reset_index()
marcasGrp.head(3)

marca,person,Asus,LG,Lenovo,Motorola,Outros,Quantum,Samsung,Sony,Xiaomi,iPad,iPhone
0,0008ed71,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
1,00091926,0.0,2.0,1.0,55.0,0.0,0.0,61.0,1.0,0.0,1.0,253.0
2,00091a7a,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0


In [14]:
#A los features ya obtenidos le agregamos las marcas de los modelos consultados
clientesGrp = clientesGrp.merge(marcasGrp,on='person')
clientesGrp.head(2)

,person,ad campaign hit,brand listing,checkout,conversion,generic listing,lead,search engine hit,searched products,staticpage,...,LG,Lenovo,Motorola,Outros,Quantum,Samsung,Sony,Xiaomi,iPad,iPhone
0,0008ed71,0.0,0.0,3.0,0.0,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
1,00091926,15.0,25.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2.0,1.0,55.0,0.0,0.0,61.0,1.0,0.0,1.0,253.0


In [ ]:
'''Seguir con feature engeneering, agregando features. Algunas ideas: 
    Modelo más consultado
    dividir el mes en 3 o 4 y ver cuantos eventos se generaron en determinadas epocas del mes
    Modelo mas consultado
    color
    almacenamiento
    navegador más usado (capaz ver los 3 o 5 más frecuentes y una columna otros)
    etc'''

## Machine learning
* En principio, usaremos random y después vamos viendo

In [16]:
X = pd.merge(clientesGrp,etiquetas,on='person')
X.head(1)

,person,ad campaign hit,brand listing,checkout,conversion,generic listing,lead,search engine hit,searched products,staticpage,...,Lenovo,Motorola,Outros,Quantum,Samsung,Sony,Xiaomi,iPad,iPhone,label
0,0008ed71,0.0,0.0,3.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,False


In [17]:
X.drop("person", axis=1, inplace=True)
X = X.fillna(0)
y = X.pop("label")

In [64]:
#Probablemente tenga que mover esto al principio
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
#import utils

In [19]:
clf = RandomForestClassifier(n_estimators=400, n_jobs=-1)
scores = cross_val_score(estimator=clf,X=X, y=y, cv=3,
                         scoring="roc_auc", verbose=True)

"AUC %.2f +/- %.2f" % (scores.mean(), scores.std())

[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:   17.4s finished


'AUC 0.82 +/- 0.01'

In [55]:
def feature_importances(model, features, n=10):
    importances = model.feature_importances_
    zipped = sorted(zip(features, importances), key=lambda x: -x[1])
    for i, f in enumerate(zipped[:n]):
        print("%d: Feature: %s, %.3f" % (i+1, f[0], f[1]))

    return [f[0] for f in zipped[:n]]

In [70]:
clf.fit(X, y)
#top_features = feature_importances(clf, features_encoded, n=20)
valores = (zip(map(lambda x: round(x, 4), clf.feature_importances_), X.columns))


In [51]:
resultSet = set(valores)
resultSet = sorted(resultSet, reverse=True)

In [52]:
print(resultSet)

[(0.1018, 'mayo'), (0.087800000000000003, 'checkout'), (0.085199999999999998, 'viewed product'), (0.070499999999999993, 'Samsung'), (0.0654, 'iPhone'), (0.064600000000000005, 'brand listing'), (0.064199999999999993, 'ad campaign hit'), (0.0608, 'generic listing'), (0.058700000000000002, 'visited site'), (0.0533, 'search engine hit'), (0.047, 'searched products'), (0.042099999999999999, 'Motorola'), (0.037600000000000001, 'abril'), (0.028400000000000002, 'marzo'), (0.020500000000000001, 'conversion'), (0.017500000000000002, 'LG'), (0.015599999999999999, 'Sony'), (0.014999999999999999, 'staticpage'), (0.0144, 'enero'), (0.014200000000000001, 'febreo'), (0.0121, 'Lenovo'), (0.0086, 'iPad'), (0.0080999999999999996, 'Asus'), (0.0037000000000000002, 'lead'), (0.0028, 'Quantum'), (0.0, 'Xiaomi'), (0.0, 'Outros')]


In [62]:
top_features = feature_importances(clf, X.columns, n=8)

1: Feature: mayo, 0.102
2: Feature: checkout, 0.088
3: Feature: viewed product, 0.085
4: Feature: Samsung, 0.070
5: Feature: iPhone, 0.065
6: Feature: brand listing, 0.065
7: Feature: ad campaign hit, 0.064
8: Feature: generic listing, 0.061


In [63]:
#top_features  = ['mayo','checkout']
X.loc[:,top_features].head(2)

,mayo,checkout,viewed product,Samsung,iPhone,brand listing,ad campaign hit,generic listing
0,6.0,3.0,0.0,1.0,1.0,0.0,0.0,1.0
1,17.0,1.0,3.0,0.0,4.0,0.0,1.0,1.0


In [65]:
#Pruebo de nuevo los random forest con los resultados obtenidos de mejores clasificadores
X_reducido = X.loc[:,top_features]
clf = RandomForestClassifier(n_estimators=400, n_jobs=-1)
scores = cross_val_score(estimator=clf,X=X_reducido, y=y, cv=3,
                         scoring="roc_auc", verbose=True)

"AUC %.2f +/- %.2f" % (scores.mean(), scores.std())

[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:   14.6s finished


'AUC 0.81 +/- 0.01'

In [66]:
clientesAClasifcar = clientesGrp.loc[~clientesGrp['person'].isin(etiquetas['person'])]

In [67]:
clientesAClasifcar.head(2)

,person,ad campaign hit,brand listing,checkout,conversion,generic listing,lead,search engine hit,searched products,staticpage,...,LG,Lenovo,Motorola,Outros,Quantum,Samsung,Sony,Xiaomi,iPad,iPhone
1,00091926,15.0,25.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2.0,1.0,55.0,0.0,0.0,61.0,1.0,0.0,1.0,253.0
2,00091a7a,1.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0


In [71]:
prediccion = clf.predict(X=clientesAClasifcar.drop('person', axis=1))

In [75]:
pd.Series(prediccion).value_counts()

False    19090
True        36
dtype: int64

In [77]:
#Hay algo de debe estar bastaaaaante mal. De por si, necesitamos valores de probabilidad... así que busco otra cosa que devuelva
prediccionProbabilistica = clf.predict_proba(X=clientesAClasifcar.drop('person', axis=1))


In [78]:
prediccionProbabilistica

array([[ 0.92  ,  0.08  ],
       [ 0.8475,  0.1525],
       [ 0.935 ,  0.065 ],
       ..., 
       [ 1.    ,  0.    ],
       [ 0.9375,  0.0625],
       [ 1.    ,  0.    ]])

In [79]:
type(prediccionProbabilistica)

numpy.ndarray

In [80]:
prediccionProbabilistica[0][0]

0.92000000000000004

In [81]:
prediccionProbabilistica[:,[1]]

array([[ 0.08  ],
       [ 0.1525],
       [ 0.065 ],
       ..., 
       [ 0.    ],
       [ 0.0625],
       [ 0.    ]])

In [93]:
(prediccionProbabilistica[:,[1]] > 0.3).sum()

606

### Algunos análisis que hicimos por problemas de performance, comprobando lo que tanto nos comentaron del apply

In [15]:
%timeit eventos['timestamp_dt'] =  pd.to_datetime(eventos['timestamp'])

10 loops, best of 3: 51.2 ms per loop


In [12]:
%timeit eventos['timestamp'] =  pd.to_datetime(eventos['timestamp'])

The slowest run took 62.21 times longer than the fastest. This could mean that an intermediate result is being cached.
1 loop, best of 3: 18.8 ms per loop


In [14]:
%timeit eventos[['marca','modelo']] = eventos['model'].dropna().str.split(' ',n=1,expand=True)

1 loop, best of 3: 8.62 s per loop


In [16]:
%timeit eventos['weekday']=eventos['timestamp_dt'].apply(lambda x: x.day_name())

1 loop, best of 3: 1min 18s per loop


In [17]:
%timeit eventos['weekday_2']=eventos['timestamp_dt'].dt.day_name()

1 loop, best of 3: 1.21 s per loop


In [18]:
#vemos si de una y otra forma obtenemos los mismos resultados
print(eventos['weekday'].head())
print(eventos['weekday_2'].head())

0    Friday
1    Friday
2    Friday
3    Friday
4    Friday
Name: weekday, dtype: object
0    Friday
1    Friday
2    Friday
3    Friday
4    Friday
Name: weekday_2, dtype: object


### Cosas a descartar pero que capaz nos de alguna idea de cómo separar los sets de datos

In [11]:
#Nos quedamos con los eventos de los que tenemos idea de si convirtieron luego
eventosConocidos = eventos.loc[eventos['person'].isin(etiquetas['person'])]

In [22]:
eventosConocidos.describe()

,timestamp,event,person,url,sku,model,condition,storage,color,skus,...,search_engine,channel,new_vs_returning,city,region,country,device_type,screen_resolution,operating_system_version,browser_version
count,1171886,1171886,1171886,94875,665336,665767,665336,665336,665336,249587,...,52829,102299,102299,102299,102299,102299,102299,102297,102299,102299
unique,857447,11,19414,236,3405,198,5,8,63,34843,...,4,7,2,1854,84,39,4,285,107,308
top,2018-05-31 00:50:28,viewed product,ffee0f18,/,2830.0,iPhone 6,Bom,16GB,Preto,"2820,6706,6720,2750,6649,7251,6663,12604,7224,...",...,Google,Paid,Returning,Unknown,Sao Paulo,Brazil,Smartphone,360x640,Windows 7,Chrome 66.0
freq,11,629233,3458,31753,4044,52268,275492,222134,160397,1338,...,52242,45632,83173,18569,28597,99160,53171,37827,22613,28256


In [18]:
len(eventosConocidos['event'] == 'viewed product')

1171886

In [12]:
eventosConocidos['event'].value_counts()

viewed product       629233
brand listing        105993
visited site         102299
ad campaign hit       94998
generic listing       80453
searched products     63887
search engine hit     52829
checkout              32539
staticpage             5660
conversion             3564
lead                    431
Name: event, dtype: int64

In [17]:
#Posiblemente no sea necesario usar esto, habrá que ver con los algoritmos que utilizaremos
#De los eventos conocidos hago un merge. Claramente podría haber hecho el merge antes, pero cuando lo probé antes, falló por 
eventosConocidosMrg = eventosConocidos.merge(etiquetas,on='person')